In [2]:
from golden import *
import pandas as pd
import numpy as np

In [3]:
df_neurotech = pd.read_csv('/mnt/c/Users/anderson.paiva/Documents/03.3 Squad Saúde/Projeto Neurotech/base_pov - maio-junho.csv')

In [4]:
df_neurotech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19993 entries, 0 to 19992
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   mes              19993 non-null  object
 1   origem           19993 non-null  object
 2   id_atendimento   19993 non-null  int64 
 3   id_procedimento  19993 non-null  int64 
 4   vl_repasse       19993 non-null  object
 5   fl_auditoria     19993 non-null  object
 6   status_novo      19993 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.1+ MB


In [5]:
# Iniciando o client para acessar o BQ
client_bq = config_client('prod')

# Verificando se o client está conectado
check_client_connection(client_bq)

# Declarando a query que vamos utilizar
query = """
select cast(id_atendimento as int64) as id_atendimento,
       numero_atendimento,
       status_novo,
       vl_bruto, -- Trazendo para análise do processo atual
       motivo_glosa_desc,
       data_origem_date,
       nome_procedimento,
       fl_edicao_procedimento,
       diff_apos_edicao,
       case when status_novo in ('Glosado Irrecursável', 'Recurso recusado', 'Fora prazo recurso') then 'Definitivo'
            when status_novo in ('Glosado Recursável', 'Recursado', 'Recurso aceito', 'Recurso aceito parcialmente') then 'Não definitiva'
       end as status_glosa
from `op_health.custos_medicos`
where sistema_origem_custo = 'ERP Nofaro'
and date_trunc(data_origem_date, month) >= '2024-05-01'
"""

query_bytes(query, client_bq)

/home/andersonxpm/.local/lib/python3.8/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Quer matar o Cesar????


'1.22 GB'

In [6]:
df_custos_medicos = run_query(query, client_bq)

Quer matar o Cesar????
Esta consulta processará 1.22 GB.


Deseja executar a consulta? (Y/N) Y


Iniciando query
Iniciando a consulta: 2024-09-03 15:06:12
Consulta concluída: 2024-09-03 15:10:16
Duração da consulta: 0:04:04.360572


In [7]:
df_custos_medicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2541831 entries, 0 to 2541830
Data columns (total 10 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   id_atendimento          int64         
 1   numero_atendimento      int64         
 2   status_novo             object        
 3   vl_bruto                float64       
 4   motivo_glosa_desc       object        
 5   data_origem_date        datetime64[ns]
 6   nome_procedimento       object        
 7   fl_edicao_procedimento  int64         
 8   diff_apos_edicao        float64       
 9   status_glosa            object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 193.9+ MB


In [8]:
df_uniao = pd.merge(df_neurotech, right = df_custos_medicos, how = 'left', left_on = 'id_procedimento', right_on = 'id_atendimento')

In [9]:
df_final = df_uniao[['mes', 'origem', 'id_atendimento_x', 'id_procedimento', 'vl_repasse', 'fl_auditoria', 'status_novo_x', 'status_novo_y']]

In [10]:
df_final

,mes,origem,id_atendimento_x,id_procedimento,vl_repasse,fl_auditoria,status_novo_x,status_novo_y
0,maio/24,PoV Indicadores,3069004,6440570,R$ 8,AUDITADO PROC ATUAL,Aprovado,Aprovado
1,maio/24,PoV Indicadores,3069004,6440567,R$ 0,AUDITADO PROC ATUAL,Aprovado,Aprovado
2,maio/24,PoV Indicadores,3069004,6440569,R$ 0,AUDITADO PROC ATUAL,Glosado Irrecursável,Glosado Irrecursável
3,maio/24,PoV Indicadores,3069004,6440568,R$ 0,AUDITADO PROC ATUAL,Glosado Irrecursável,Glosado Irrecursável
4,maio/24,PoV Indicadores,3069231,6441074,R$ 16,AUDITADO PROC ATUAL,Aprovado,Aprovado
...,...,...,...,...,...,...,...,...
19988,junho/24,PoV Indicadores,3346783,7018503,R$ 22,AUDITADO PROC ATUAL,Recursado,Recurso recusado
19989,junho/24,PoV Indicadores,3435626,7202200,R$ 111,AUDITADO PROC ATUAL,Recursado,Recurso recusado
19990,junho/24,PoV Indicadores,3435420,7201763,R$ 114,AUDITADO PROC ATUAL,Recursado,Recurso recusado
19991,junho/24,PoV Indicadores,3596849,7533274,R$ 52,AUDITADO PROC ATUAL,Glosado Irrecursável,Glosado Irrecursável


In [11]:
save_to_csv(df_final, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'base_pov_status_final_20240903')

'Arquivo base_pov_status_final_20240903.csv salvo com sucesso em /mnt/c/Users/anderson.paiva/Downloads/base_pov_status_final_20240903.csv!!'

In [11]:
# Agora vamos criar a coluna com as faixas de score (Categorização do score)
condicoes = [
    (df_uniao['SCORE']<= .1),
    (df_uniao['SCORE']>.1) & (df_uniao['SCORE']<=.2),
    (df_uniao['SCORE']>.2) & (df_uniao['SCORE']<=.3),
    (df_uniao['SCORE']>.3) & (df_uniao['SCORE']<=.4),
    (df_uniao['SCORE']>.4) & (df_uniao['SCORE']<=.5),
    (df_uniao['SCORE']>.5) & (df_uniao['SCORE']<=.6),
    (df_uniao['SCORE']>.6) & (df_uniao['SCORE']<=.7),
    (df_uniao['SCORE']>.7) & (df_uniao['SCORE']<=.8),
    (df_uniao['SCORE']>.8) & (df_uniao['SCORE']<=.9),
    (df_uniao['SCORE']>.9)
]

categorias = [
    '0 - 0 a 10%',
    '1 - 10 a 20%',
    '2 - 20 a 30%',
    '3 - 30 a 40%',
    '4 - 40 a 50%',
    '5 - 50 a 60%',
    '6 - 60 a 70%',
    '7 - 70 a 80%',
    '8 - 80 a 90%',
    '9 - 90 a 100%',
]
df_uniao['CLASSIFICACAO_SCORE'] = np.select(condicoes, categorias, np.nan)
df_uniao.info()

KeyError: 'SCORE'

In [11]:
df_uniao = df_uniao.drop(['id_atendimento', 'numero_atendimento', 'data_origem_date', 'nome_procedimento'], axis = 1)

In [12]:
df_uniao = df_uniao.rename(columns = {'PREV ANALISADO': 'FL_JA_ANALISADO', 'status_novo': 'STATUS', 'vl_bruto': 'VL_REPASSE', 'motivo_glosa_desc': 'MOTIVO_GLOSA', 'fl_edicao_procedimento':'FL_EDICAO_PROCEDIMENTO', 'diff_apos_edicao': 'DIFF_APOS_EDICAO', 'status_glosa':'STATUS_GLOSA'})

In [14]:
df_uniao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25052 entries, 0 to 25051
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SCORE                   25052 non-null  float64
 1   ID_EVENTO               25052 non-null  int64  
 2   ID_ATENDIMENTO          25052 non-null  int64  
 3   DATA_SOLICITACAO        25052 non-null  object 
 4   STATUS                  25052 non-null  object 
 5   VL_REPASSE              25052 non-null  float64
 6   MOTIVO_GLOSA            3803 non-null   object 
 7   FL_EDICAO_PROCEDIMENTO  25052 non-null  int64  
 8   DIFF_APOS_EDICAO        25039 non-null  float64
 9   STATUS_GLOSA            3801 non-null   object 
 10  CLASSIFICACAO_SCORE     25052 non-null  object 
dtypes: float64(3), int64(3), object(5)
memory usage: 2.1+ MB


In [15]:
save_to_csv(df_uniao, path = '/mnt/c/Users/anderson.paiva/Downloads', file_name = 'base_poc_com_class_score')

'Arquivo base_poc_com_class_score.csv salvo com sucesso em /mnt/c/Users/anderson.paiva/Downloads/base_poc_com_class_score.csv!!'

------------------------